# 📦 Step-by-Step Cleaning Code

##  1. Import & Load the Data

In [1]:
import pandas as pd

# Load the raw dataset
df = pd.read_csv('../data/raw_data/apartment_cost_list.csv')
df.head()


,Job #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Community - Board,Curb Cut,Fully Permitted,Initial Cost,Proposed Zoning Sqft,Horizontal Enlrgmt,Vertical Enlrgmt,Enlargement SQ Footage,Zoning Dist1,Job Description
0,123768976,MANHATTAN,1598,2ND AVENUE,1545,49,1049907,A2,108,NaN,NaN,"$65,000.00",0,NaN,NaN,0,C1-9,INTERIOR RENOVATION TO COMBINE APARTMENT 2E AN...
1,140922893,MANHATTAN,160,E 89 ST,1517,43,1048094,A2,108,NaN,7/14/2020,"$85,000.00",0,NaN,NaN,0,R8B,HEREBY FILING APPLICATION FOR THE INTERIOR REN...
2,301773579,BROOKLYN,65,GATES AVENUE,1962,86,3056039,A1,302,NaN,5/23/2007,"$60,000.00",0,NaN,NaN,0,R6,Change existing four story and basement brick ...
3,301976057,BROOKLYN,1401,OCEAN AVENUE,7584,35,3206621,A2,314,NaN,2/28/2006,"$75,000.00",0,NaN,NaN,0,NaN,COMBINING OF APARTMENT 4A &amp; 4L.NO CHANGEIN...
4,301989276,BROOKLYN,154,NOBLE STREET,2573,10,3065099,A1,301,NaN,9/29/2006,"$40,000.00",0,NaN,NaN,0,C4-3,New C of O for existing building. New duplex a...


##  2. Check Data Info

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19613 entries, 0 to 19612
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Job #                   19613 non-null  int64 
 1   Borough                 19613 non-null  object
 2   House #                 19613 non-null  object
 3   Street Name             19613 non-null  object
 4   Block                   19613 non-null  int64 
 5   Lot                     19613 non-null  int64 
 6   Bin #                   19613 non-null  int64 
 7   Job Type                19613 non-null  object
 8   Community - Board       19613 non-null  int64 
 9   Curb Cut                16 non-null     object
 10  Fully Permitted         15625 non-null  object
 11  Initial Cost            19613 non-null  object
 12  Proposed Zoning Sqft    19613 non-null  int64 
 13  Horizontal Enlrgmt      163 non-null    object
 14  Vertical Enlrgmt        242 non-null    object
 15  En

📊 General Shape:
- Total rows: 19,613
- Total columns: 18
- Memory usage: ~2.7 MB

### 📋 Column-Wise Insights

| Column Name                | Non-Null Count | What It Means                                                    |
|----------------------------|----------------|------------------------------------------------------------------|
| Job #                      | 19,613         | Fully populated — unique ID per renovation filing               |
| Borough                    | 19,613         | Good — every record is associated with a borough                |
| House #                    | 19,613         | All present — used for address matching                         |
| Street Name                | 19,613         | All present — part of full address                              |
| Block, Lot, Bin #          | 19,613         | Good — tax lot/location info fully available                    |
| Job Type                   | 19,613         | Fully available — critical for segmentation (A1/A2/A3)          |
| Community - Board          | 19,613         | Fully available — good for neighborhood analysis                |
| Curb Cut                   | 16             | 💥 Mostly null — not useful for analysis                        |
| Fully Permitted            | 15,625         | ❗ ~20% missing — affects time-series modeling                   |
| Initial Cost               | 19,613         | Fully present — but needs to be converted to float              |
| Proposed Zoning Sqft       | 19,613         | Present — may help zoning analytics                             |
| Horizontal/Vertical Enlrgmt| Very sparse    | 💥 Mostly null — optional to drop                               |
| Enlargement SQ Footage     | 19,613         | Mostly 0 — check if useful                                      |
| Zoning Dist1               | ~17,517        | Some missing — useful for zoning-type grouping                 |
| Job Description            | 19,613         | Full — key for NLP and job-type classification                  |


## 3. Clean Initial Cost Column

In [3]:
# Check unique formats first
df['Initial Cost'].sample(5)

6791     $59,000.00 
2577     $63,000.00 
18552    $60,000.00 
3601     $68,000.00 
8476     $56,000.00 
Name: Initial Cost, dtype: object

In [5]:
# ✅ Clean Initial Cost column: remove $, commas, and convert to float
df['Initial Cost'] = df['Initial Cost'].replace('[\$,]', '', regex=True).astype(float)

# 🧪 Quick check
df[['Initial Cost']].describe()

,Initial Cost
count,19613.000000
mean,59442.155713
std,26687.304059
min,40000.000000
25%,45000.000000
50%,57000.000000
75%,70000.000000
max,490000.000000


### ✅ What This Does
- Removes dollar signs and commas from the `Initial Cost` column using regular expressions.
- Converts the cleaned string values into numeric floats.
- Shows summary statistics like mean, min, and max using `.describe()` to verify the conversion.

# 4. Convert Fully Permitted to Datetime

In [6]:
# ✅ Convert Fully Permitted to datetime
df['Fully Permitted'] = pd.to_datetime(df['Fully Permitted'], errors='coerce')

# 🧪 Check for nulls created during conversion
df['Fully Permitted'].isna().sum()


3988

### ✅ What This Does
- Converts the `Fully Permitted` column into proper datetime format using `pd.to_datetime()`.
- Rows with invalid date strings will be converted to `NaT` (Not a Time).
- We use `errors='coerce'` to avoid crashing on bad data.
- Displays how many rows have missing dates after conversion.


## 5. Normalize Borough

In [7]:
# ✅ Normalize Borough column (title case and strip whitespace)
df['Borough'] = df['Borough'].str.title().str.strip()

# 🧪 Check unique boroughs
df['Borough'].unique()


array(['Manhattan', 'Brooklyn', 'Bronx', 'Queens', 'Staten Island'],
      dtype=object)

### ✅ What This Does
- Converts all borough names to Title Case (e.g., 'MANHATTAN' → 'Manhattan').
- Strips any leading or trailing whitespaces.
- Displays the list of unique borough values to confirm standardization.


## 6. Drop Rows with Missing Critical Values

In [8]:
# ✅ Drop rows missing essential fields
df_cleaned = df.dropna(subset=['Initial Cost', 'Job Type', 'Fully Permitted'])

# 🧪 Show shape before vs after
print("Before Cleaning:", df.shape)
print("After Cleaning:", df_cleaned.shape)


Before Cleaning: (19613, 18)
After Cleaning: (15625, 18)


### ✅ What This Does
- Removes rows where any of the following are missing: `Initial Cost`, `Job Type`, or `Fully Permitted`.
- These columns are required for meaningful analysis, trend segmentation, and modeling.
- Displays shape of the DataFrame before and after cleaning to confirm how many rows were removed.


## 7. Save Cleaned Data

In [9]:
# ✅ Save cleaned data to the processed_data folder
output_path = '../data/processed_data/apartment_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)

print("✅ Cleaned data saved to:", output_path)


✅ Cleaned data saved to: ../data/processed_data/apartment_cleaned.csv


### ✅ What This Does
- Saves the final cleaned dataset to `data/processed_data/apartment_cleaned.csv`.
- This version will be used in all future steps like EDA and ML modeling.
- Keeps raw and processed data separate for version control and reproducibility.